In [36]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import jieba
import nltk
from collections import OrderedDict
%matplotlib inline

In [2]:
risk_qa = pd.read_excel("risk_data.xlsx")

In [3]:
risk_qa.head()

,客戶編號,風險等級,風險等級分類,適性評量填寫日期,適性版號,題目序號,KYC_QUESTION,ANS_ORDER,ANS_ITEM,答題得分,客戶職業與任職部門,AGENT代碼
0,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A0,是否為其他國家稅務居住者?,1,1.否,0,NaN,9999999999
1,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A01,職業／行業別,8,8.退休人士,0,NaN,9999999999
2,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A02,職務,1,1.基層職員,0,NaN,9999999999
3,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A03,任職單位,1,1.任職公司／機構,0,寶豐,9999999999
4,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A03,任職單位,2,2.任職部門,0,營建,9999999999


In [4]:
uni_id = np.unique(risk_qa["客戶編號"])

In [5]:
attr_dict = defaultdict(lambda: defaultdict(list))
max_ans_dict = defaultdict(int) # record max answer num for each question (for one-hot array)

for index, row in risk_qa.iterrows():
    attr_dict[row["客戶編號"]][row["題目序號"]].append(row["ANS_ORDER"])
    if max_ans_dict[row["題目序號"]]<row["ANS_ORDER"]:
        max_ans_dict[row["題目序號"]] = row["ANS_ORDER"]


In [11]:
for k1 in attr_dict.keys():
    for k2 in attr_dict[k1].keys():
        attr_dict[k1][k2] = list(set(attr_dict[k1][k2]))

In [14]:
# 問卷答案的字典
attr_dict

defaultdict(<function __main__.<lambda>()>,
            {261909: defaultdict(list,
                         {'A0': [1],
                          'A01': [8],
                          'A02': [1],
                          'A03': [1, 2],
                          'A04': [4],
                          'A05': [3],
                          'A06': [1],
                          'A07': [1],
                          'A08': [1, 3, 5],
                          'A09': [2, 3],
                          'A10': [2],
                          'A11': [1],
                          'A12': [1],
                          'A13': [1],
                          'A14': [4],
                          'B01': [3],
                          'B02': [1, 2, 3, 4, 5],
                          'B03': [3, 4, 5],
                          'B04': [5],
                          'B05': [4],
                          'B06': [4],
                          'B07': [4]}),
             269963: defaultdict(list,
           

#### 每個答案做成一個字 ， 每個 user 當一篇文章，計算答案的共現性


In [32]:
doc_word_list = []
for uid in attr_dict.keys():
    single_word_list = []
    for aid in attr_dict[uid].keys():
        single_word_list += [aid + "_" + str(ans) for ans in attr_dict[uid][aid]]
    doc_word_list.append(single_word_list)

In [37]:
names = list(set([w for l in doc_word_list for w in l]))
occurrences = OrderedDict((name, OrderedDict((name, 0) for name in names)) for name in names)
 
# Find the co-occurrences:
for l in doc_word_list:
    for i in range(len(l)):
        for item in l[:i] + l[i + 1:]:
            occurrences[l[i]][item] += 1
            
co_occur = pd.DataFrame.from_dict(occurrences)

In [40]:
co_occur

,B02_2,B07_5,B06_1,A02_4,A01_22,A06_2,A11_1,A02_6,A12_2,A08_4,...,A01_9,B03_3,A04_5,B06_3,B03_5,A10_4,A02_1,A08_7,A10_5,A12_1
A01_1,101,87,0,26,0,4,76,0,39,50,...,0,99,111,54,154,11,100,1,1,173
A01_10,1,0,0,0,0,0,2,0,0,0,...,0,3,0,3,1,0,3,0,0,3
A01_12,12,15,1,7,0,0,16,2,6,2,...,0,9,8,6,15,2,6,0,0,19
A01_13,18,16,0,12,0,1,7,5,6,9,...,0,18,2,13,31,5,15,0,0,28
A01_14,10,12,0,1,0,0,9,3,3,4,...,0,8,11,4,19,2,1,0,1,21
A01_17,7,5,0,3,0,0,9,0,3,4,...,0,10,4,2,13,2,7,0,0,13
A01_18,2,2,0,0,0,0,0,1,0,1,...,0,1,1,1,4,1,0,0,1,3
A01_19,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
A01_2,59,50,1,29,0,0,37,6,22,22,...,0,61,57,27,92,13,18,0,3,93
A01_20,1,0,0,0,0,0,4,0,0,2,...,0,3,0,3,1,0,0,0,0,4


In [6]:
question_list = list(max_ans_dict.keys())

In [ ]:
# create feature matrix (one-hot encoding)
feature_matrix = []
id_list = []
for user in attr_dict.keys():
    single_user_feature = []
    for q in question_list:
        ans = np.array(attr_dict[user][q])-1
        one_hot = np.zeros(max_ans_dict[q])
        if len(ans)!=0:
            one_hot[ans] = 1
        single_user_feature.append(one_hot)
    merge_feature = np.hstack(single_user_feature)
    feature_matrix.append(merge_feature)
    id_list.append(user)
feature_matrix = np.vstack(feature_matrix)

In [19]:
# 客戶之風險等級
risk_dist = risk_qa[["客戶編號","風險等級"]].drop_duplicates()["風險等級"]

In [29]:
np.unique(risk_dist, return_counts=True)

(array([2, 4, 5], dtype=int64), array([   2,  101, 1786], dtype=int64))

In [ ]:
feature_matrix.shape

In [ ]:
X = feature_matrix
X_embedded = TSNE(n_components=2, perplexity=80).fit_transform(X)
X_embedded = PCA(n_components=2).fit_transform(X)

X_embedded.shape

In [ ]:
plt.figure(figsize=(16,9))
# cm = plt.cm.get_cmap("tab20", 18)
sc = plt.scatter(X_embedded[:,0], X_embedded[:,1])
plt.show()

# 實際投資行為

In [ ]:
invest = pd.read_excel("invest_data.xlsx")

In [ ]:
invest["weight"] = invest["風險等級"].apply(lambda x:int(x[-1]))
invest["weight_amount"] = invest["weight"]*invest["金額"]

In [ ]:
customer = list(set(invest["客戶"]))
customer_count = invest.groupby('客戶',as_index=False, sort=False).count()
customer_total = invest.groupby('客戶',as_index=False, sort=False).sum()
customer_total.head()

In [ ]:
customer_total["weight_level"] = customer_total["weight_amount"]/customer_total["金額"]

In [ ]:
customer_total.head()

In [ ]:
plt.hist(customer_total["金額"], bins=50)
# plt.xlim(0,50000)
plt.title("investment")
plt.show()

In [ ]:
sum(customer_money["金額"]<400000)/len(customer_money)

In [ ]:
customer_money.loc[customer_money["客戶"]==c, "風險等級"].values[0]

In [ ]:
vis_vec = []
amount = []
risk_level = []
for c in customer:
    money = customer_total.loc[customer_total["客戶"]==c, "金額"].values[0]
    level = customer_total.loc[customer_total["客戶"]==c, "weight_level"].values[0]
    if money<100000:
        vis_vec.append(X_embedded[id_list.index(c)])
        amount.append(money)
        risk_level.append(level)
vis_vec = np.vstack(vis_vec)

In [ ]:
plt.hist(risk_level,bins=40)
plt.title("weighted risk level")
plt.show()

In [ ]:
amount = (np.array(amount)-min(amount))/(max(amount)-min(amount))

In [ ]:
cmap = sns.cubehelix_palette(as_cmap=True)

f, ax = plt.subplots(figsize=(16,9))
points = ax.scatter(vis_vec[:,0], vis_vec[:,1], c=risk_level, s=50, cmap=cmap)
f.colorbar(points)
plt.title("risk level")
plt.show()

In [ ]:
cmap = sns.cubehelix_palette(as_cmap=True)

f, ax = plt.subplots(figsize=(16,9))
points = ax.scatter(vis_vec[:,0], vis_vec[:,1], c=amount, s=50, cmap=cmap)
f.colorbar(points)
plt.title("investment amount")
plt.show()

# 產品名稱

In [ ]:
product_corpus = "".join(list(invest["基金簡稱"]))

In [ ]:
jieba.set_dictionary('C:/Users/thtang/Desktop/dict.txt.big.txt')

In [ ]:
product_words = jieba.lcut(product_corpus)

In [ ]:
" ".join(product_words).replace("基金","").replace("野村","")

In [ ]:
freq = nltk.FreqDist(product_words)

In [ ]:
freq.most_common()